In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_counts = application_df["APPLICATION_TYPE"].value_counts()
app_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_counts[app_counts < 250].keys()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_counts = application_df["CLASSIFICATION"].value_counts()
class_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_counts[class_counts < 1000].keys()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [15]:
# Since the monetary values are inherently ordered in their magnitude, and get_dummies can't capture that, I'm going to map them onto ordered integers instead
income_mapping = {
    '0': 0,
    '1-9999': 1,
    '10000-24999': 2,
    '25000-99999': 3,
    '100000-499999': 4,
    '1M-5M': 5,
    '5M-10M': 6,
    '10M-50M': 7,
    '50M+': 8
}
application_df["INCOME_ENCODED"] = application_df["INCOME_AMT"].map(income_mapping)
application_df = application_df.drop("INCOME_AMT", axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,INCOME_ENCODED
0,T10,Independent,C1000,ProductDev,Association,1,N,5000,1,0
1,T3,Independent,C2000,Preservation,Co-operative,1,N,108590,1,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,N,5000,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,N,6692,1,2
4,T3,Independent,C1000,Heathcare,Trust,1,N,142590,1,4


In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df, columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "SPECIAL_CONSIDERATIONS"])

In [37]:
# Split our preprocessed data into our features and target arrays
y = app_dummies["IS_SUCCESSFUL"].values
X = app_dummies.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [39]:
len(X_train[0])

35

In [47]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 32
hidden_nodes_layer3 = 16
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)
# LeakyReLU layer
nn.add(
    tf.keras.layers.LeakyReLU(alpha=0.01)
)
# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu")
)
# LeakyReLU layer
nn.add(
    tf.keras.layers.LeakyReLU(alpha=0.01)
)
# Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu")
)
# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)
# Check the structure of the model
nn.summary()
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\sicke\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sicke\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_17 (Dense)                     │ (None, 128)                 │           4,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 4)                   │              68 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,337 (36.47 KB)

 Trainable params: 9,337 (36.47 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# Compile the model
nn.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

In [51]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 967us/step - accuracy: 0.6912 - loss: 0.6237
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - accuracy: 0.7295 - loss: 0.5721
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 964us/step - accuracy: 0.7256 - loss: 0.5640
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 988us/step - accuracy: 0.7224 - loss: 0.5625
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 966us/step - accuracy: 0.7363 - loss: 0.5500
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 954us/step - accuracy: 0.7303 - loss: 0.5553
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 975us/step - accuracy: 0.7286 - loss: 0.5552
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - accuracy: 0.7322 - loss: 0.5499
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 964us/step - accuracy: 0.7303 - loss: 0.5556
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 974us/step - accuracy: 0.7286 - loss: 0.5525
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 985us/step - accuracy: 0.7356 - loss: 0.5486
Epoch 12/20
804/804 ━━━━━━━━━━